In [3]:
import tensorflow as tf

<module 'tensorflow._api.v2.version' from 'C:\\Users\\paulc\\Desktop\\Imperial College\\Spring term\\Deep Learning\\venv\\lib\\site-packages\\tensorflow\\_api\\v2\\version\\__init__.py'>
